# GIS Visualization of Building Footprint
This notebook shows how to use C# and Python to visualize a building on a map.
Make sure the model has a survey coordinate system defined with real world coordinates.

Installation of Jupyter is required. See [here](https://github.com/dotnet/interactive/blob/main/docs/jupyter-in-polyglot-notebooks.md) for options.

All python-dependencies are stored in requirements.txt.
Create a python cell and run below to install:
> pip install -r requirements.txt

Run below command to connect to a Python-kernel:

In [1]:
#!connect jupyter --kernel-name pythonkernel --kernel-spec python3

The `#!connect jupyter` feature is in preview. Please report any feedback or issues at https://github.com/dotnet/interactive/issues/new/choose.

Kernel added: #!pythonkernel

Once connected, we setup connection to a revit interactive kernel. Run below:

In [2]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#r "nuget:RevitInteractive"

In [ ]:
#!connect revit --kernel-name revit --revit-version 2024

In [ ]:
#r "nuget:Revit.RevitApi.x64, 2023.0.0"
#r "nuget:Revit.RevitApiUi.x64, 2023.0.0"
using Autodesk.Revit.DB;
using Autodesk.Revit.UI;

  2 standard systems of units exists: the international one (SI), also known as the metric system, and the imperial system.
  Internally, Revit stores in the imperial system.

In [ ]:
#!revit
// Assume 'doc' is the current Revit Document you are working with
Units units = doc.GetUnits();
// Get the FormatOptions for a specific UnitType (e.g., Length)
FormatOptions lengthFormatOptions = units.GetFormatOptions(SpecTypeId.Length);

// Get the ForgeTypeId for the Length unit
ForgeTypeId lengthUnitTypeId = lengthFormatOptions.GetUnitTypeId();

// Output the ForgeTypeId to the console or use it as needed
lengthUnitTypeId

## Shared Coordinates

The project location defines the offset from the site location and mesasures in decimal feet.

For each location there are two basepoints: project base boint and survey point.

We will use the currently active project location. 

In [ ]:
#!revit
var projectBasePoint = new FilteredElementCollector(doc).OfClass(typeof(BasePoint))
                                                  .Cast<BasePoint>()
                                                  .Where(x => x.IsShared!)
                                                  .First();

var surveyPoint = new FilteredElementCollector(doc).OfClass(typeof(BasePoint))
                                                  .Cast<BasePoint>()
                                                  .Where(x => x.IsShared)
                                                  .First();


string prompt1 = "Displays survey and base point in the active ProjectLocation:\n";
prompt1 +=  "\n\t" + $"Survey point:{surveyPoint.SharedPosition}";
prompt1 +=  "\n\t" + $"Project base point:{projectBasePoint.SharedPosition}";
display(prompt1);


string prompt2 = "Displays survey and base point in internal coordinate system:\n";
prompt2 += "\n\t" +  $"Survey point:{surveyPoint.Position}";
prompt2 += "\n\t" + $"Project base point:{projectBasePoint.Position}";
display(prompt2);

ProjectLocation projectLocation = doc.ActiveProjectLocation;

ProjectPosition projectPosition = projectLocation.GetProjectPosition(new XYZ(0,0,0));

projectPosition

In [ ]:

#!revit

var convertToDisplayUnits = (double valueInFeet) => UnitUtils.ConvertFromInternalUnits(valueInFeet, lengthUnitTypeId);

// Format the prompt string to show the message.
String prompt = "Current project location information converted to display units:\n";
prompt += "\n\t" + "Origin point position:";
prompt += "\n\t\t" + "Angle: " + projectPosition.Angle;
prompt += "\n\t\t" + "East to West offset: " + convertToDisplayUnits(projectPosition.EastWest); //The offset of the project location from the site location in the east/west direction. Measured in decimal feet.
prompt += "\n\t\t" + "Elevation: " + convertToDisplayUnits(projectPosition.Elevation);
prompt += "\n\t\t" + "North to South offset: " + convertToDisplayUnits(projectPosition.NorthSouth);
display(prompt);


XYZ translationVector = new XYZ(
  projectPosition.EastWest,
  projectPosition.NorthSouth,
  projectPosition.Elevation);

Transform translationTransform
  = Transform.CreateTranslation(
    translationVector);

// Create a rotation for the angle about true north

Transform rotationTransform
  = Transform.CreateRotationAtPoint(XYZ.BasisZ, projectPosition.Angle, XYZ.Zero);
// Combine the transforms 

Transform finalTransform
  = translationTransform.Multiply(
    rotationTransform);

var internalToSurveyTransform = finalTransform;


display(internalToSurveyTransform.OfPoint(new XYZ(0,0,0)));
// Use the inverse of the survey point transform to convert the point from internal to survey coordinates
internalToSurveyTransform

Filter all floors in active view. If you leave active view out in the collector, it will search the entire database.
Also collect solids of the floors and return the list. 

In [ ]:
#!revit


var floorsInActiveView = new FilteredElementCollector(doc, doc.ActiveView.Id)
                .WhereElementIsNotElementType()
                .WhereElementIsViewIndependent()
                .OfClass(typeof(Floor))
                .OfType<Floor>();


/*
var allFloors = new FilteredElementCollector(doc)
                .WhereElementIsNotElementType()
                .WhereElementIsViewIndependent()
                .OfClass(typeof(Floor))
                .OfType<Floor>();


var floorsNotInActiveView = allFloors.Where(f => !floorsInActiveView.Any(x => x.Id.IntegerValue == f.Id.IntegerValue));
*/


//display($"Floors not in active view: {floorsNotInActiveView.Count()}");
//display(floorsNotInActiveView.First().Id);

List<Solid> solids = new List<Solid>();
foreach (var floor in floorsInActiveView){

    GeometryElement geometry = floor.get_Geometry(new Options());

    foreach(var g in geometry){

        Solid solid = g as Solid;

        if (solid != null){
            solids.Add(solid);
        }

    }

}
display(solids.Count());
solids
                

In each solid, we pick the lowest face.

In [ ]:
#!revit

List<PlanarFace> planarFaces = new List<PlanarFace>();
foreach (var solid in solids){

PlanarFace lowest = null;
    FaceArray faceArray = solid.Faces;
    foreach( Face f in faceArray )
    {
      PlanarFace pf = f as PlanarFace;
      if( null != pf && (pf.FaceNormal.Z==1.0 || pf.FaceNormal.Z==-1.0))
      {
        if( ( null == lowest )
          || ( pf.Origin.Z < lowest.Origin.Z ) )
        {
          lowest = pf;
        }
      }
    }
planarFaces.Add(lowest);

}
planarFaces

Because the transform will give us coordinates in imperial units we need to convert to display settings if metric system is used.

In [ ]:
#!revit

var convertToDisplayUnits = (XYZ pt) => new XYZ(UnitUtils.ConvertFromInternalUnits(pt.X, lengthUnitTypeId),
                                                UnitUtils.ConvertFromInternalUnits(pt.Y, lengthUnitTypeId),
                                                UnitUtils.ConvertFromInternalUnits(pt.Z, lengthUnitTypeId));

List<List<XYZ>> polygons = new List<List<XYZ>>();

display(planarFaces.Count());

foreach (PlanarFace f in planarFaces){
//display(f.EdgeLoops);
    if (f!=null){
    var loops = f.EdgeLoops;
    foreach (EdgeArray loop in loops)
        {
            // Select many flattens the list of lists of points into a single list of points
            List<XYZ> vertices = loop.Cast<Edge>()
                .SelectMany(e => e.Tessellate(), (e, point) => convertToDisplayUnits(internalToSurveyTransform.OfPoint(point)) )
                .ToList();

            // Remove last point if it's a duplicate of the first (closed loop assumption)
            if (vertices.First().IsAlmostEqualTo(vertices.Last()))
            {
                vertices.RemoveAt(vertices.Count - 1);
            }

            polygons.Add(vertices);
        }
    }
}

polygons


Cell below sends the polygon data to the python-kernel for processing.

In [ ]:
#!set --value @revit:polygons --name polygons

Extract the XY-coordinates and build polygons. Order it by area and pick the largest one. 

In [ ]:
import shapely
from shapely.geometry import Point, Polygon, LineString, GeometryCollection
from shapely.ops import unary_union
import numpy as      np

coords = list(map(lambda p: list(map(lambda pt: (pt["X"],pt["Y"]),p) ),polygons))

_polygons = list(map(lambda coords: Polygon(coords),coords))
_sorted = sorted(_polygons, key=lambda p: p.area, reverse=True)
footprint = _sorted[0]

Execute cell below to produce a shape-file in same directory as this notebook.

You can use QGIS or other GIS-software to visualize the shape on a map.

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
from fiona.crs import from_epsg

# Create an empty geopandas GeoDataFrame
newdata = gpd.GeoDataFrame()

# Insert the polygon into 'geometry' -column at index 0
newdata.loc[0, 'geometry'] = footprint

# Set the GeoDataFrame's coordinate system to WGS84 (i.e. epsg code 4326)
# https://spatialreference.org/ref/epsg/2272/
#newdata.crs = from_epsg(3007)
newdata.crs = from_epsg(2272) #Revit 2024 Snowdon Towers CS

# Determine the output path for the Shapefile
outfp = "footprint.shp"

# Write the data into that Shapefile
newdata.to_file(outfp)